# Spliting Data Set Into Sentences
The idea is to split each file into it's sentences and to filter them for single keywords, to reduce noise. In addition, a context window can be specified, which keeps the sentences arround the filtered sentence. 

In [56]:
import os
import re
from nltk.tokenize import sent_tokenize

## Global Information

In [57]:
dir_path = "/Users/Daniel/Downloads/final_datasets/full_dataset"
out_dir = "daniel_181218_2"

In [58]:
clustering_lst = ["clustering is"]
pm_lst = ["pattern mining", "sequence mining", "association rule", "sequential pattern"]
prediction_lst = ["regression is", "classification is"]
#association_lst = ["association rule mining is", "pattern", "sequences", "mining"]

In [59]:
keyword_lst = {"clustering": clustering_lst,
               "prediction": prediction_lst,
               "frequent_pattern_mining": pm_lst}

In [60]:
context_window = 1 # bidirectional 

In [61]:
if not os.path.isdir(out_dir):
    os.mkdir(out_dir)
    print("dir created")

dir created


## Reading, Splitting and Saving Files

In [62]:
%%time
for root, dirs, files in os.walk(dir_path):
    for _dir in dirs: 
        for txt_file in [x for x in os.listdir(os.path.join(root, _dir)) if x.endswith((".txt", ".TXT"))]:
            # Class name = dir name
            class_name = _dir
            #Read File
            file_name = os.path.join(root, _dir, txt_file)
            file = open(file_name, "r")
            txt = file.read()
            file.close()
            # Txt to List[Sentences]
            sentences = sent_tokenize(txt)
            # Abstracts
            for i, sentence in enumerate(sentences):
                keywords = keyword_lst[class_name]
                for keyword in keywords:
                    if keyword in sentence:
                        text = []
                        main_sentence = sentence # the sentence in the middle of the context window
                        # getting the context
                        if i != 0:
                            # check if list is big enough for a bidirectional check
                            if (i-context_window) >= 0 and (i+context_window+1) <= len(sentences):
                                left_context = i-context_window
                                right_context = i+context_window
                                text = sentences[left_context:(right_context+1)]
                            # Only space left on the left side
                            elif (i-context_window) >= 0: 
                                left_context = i-context_window
                                text = sentences[left_context:(i+1)]
                            # only space at the right side
                            elif len(sentences) <= (i+context_window+1) and i+1 != len(sentences):
                                right_context = i+context_window
                                text = sentences[i: (right_context+1)]
                        else:
                            text.append(main_sentence)
                        
                        # create dirs
                        if not os.path.isdir(os.path.join(out_dir, _dir)):
                            os.mkdir(os.path.join(out_dir, _dir))
                            #print("dir created")
                        
                        # write file
                        file_name_out = os.path.join(out_dir, _dir, txt_file)
                        file_out = open(file_name_out, "w+")
                        for i, txt in enumerate(text):
                            txt = re.sub(' +',' ',txt).lower().lstrip(" ").lstrip("\n")
                            txt_split = txt.split(" ", 1)
                            if txt_split[0] == "abstract":
                                txt = txt_split[1]
                            if txt_split[0].isalpha():
                                if i > 0:
                                    file_out.write(" "+txt.lower())
                                else:
                                    file_out.write(txt.lower())
                        file_out.close()

CPU times: user 15min 5s, sys: 28.7 s, total: 15min 34s
Wall time: 16min 11s
